# [road address기반 coordinate 확보]

## 1번째 작업 : geopy기반 GeoCoding

In [1]:
from geopy.geocoders import Nominatim

# Nominatim 객체 생성
geolocator = Nominatim(user_agent='South Korea', timeout=None)

# 위도/경도 반환 함수
def geocoding(address):
    try:
        geo = geolocator.geocode(address)
        x, y = [geo.latitude, geo.longitude] if geo else [None, None]
        return x, y
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None, None

In [2]:
import pandas as pd

df = pd.read_csv('I-3. merged_df_processed.csv', encoding='utf-8-sig')
df.head()
df.shape

(11300, 12)

In [3]:
# df의 '시군구' 컬럼과 '도로명'컬럼을 합쳐서 'address'컬럼 생성
df['address'] = df['시군구'] + ' ' + df['도로명']
df.head()

,시군구,번지,주택유형,도로조건,연면적(㎡),대지면적(㎡),계약년월,계약일,거래금액(만원),건축년도,도로명,비고,address
0,경상남도 창원성산구 가음동,3*,다가구,12m미만,231.78,255.2,201508,25,"53,000",1997.0,가양로52번길,2. 성산구_단독다가구(매매),경상남도 창원성산구 가음동 가양로52번길
1,경상남도 창원성산구 가음동,4*,다가구,12m미만,236.05,257.4,201606,27,"53,000",1999.0,가음로92번길,2. 성산구_단독다가구(매매),경상남도 창원성산구 가음동 가음로92번길
2,경상남도 창원성산구 가음동,*,다가구,12m미만,381.05,268.7,201604,14,"70,000",2013.0,가음정로23번길,2. 성산구_단독다가구(매매),경상남도 창원성산구 가음동 가음정로23번길
3,경상남도 창원성산구 가음동,*,다가구,12m미만,265.88,269.7,201605,16,"65,000",2016.0,가음정로11번길,2. 성산구_단독다가구(매매),경상남도 창원성산구 가음동 가음정로11번길
4,경상남도 창원성산구 가음동,4*,단독,12m미만,350.01,248.0,201509,24,"60,000",1997.0,가양로34번길,2. 성산구_단독다가구(매매),경상남도 창원성산구 가음동 가양로34번길


In [4]:
# df의 '시군구'에서 공백을 기준으로 도 / 시(군/구) / 동을 분리
# 예: '경상남도 창원성산구 가음동' -> ['경상남도', '창원성산구', '가음동']
divided_df = df['시군구'].str.split(n=2, expand=True)
divided_df.columns = ['도', '시군구', '읍면동']
divided_df.head()

,도,시군구,읍면동
0,경상남도,창원성산구,가음동
1,경상남도,창원성산구,가음동
2,경상남도,창원성산구,가음동
3,경상남도,창원성산구,가음동
4,경상남도,창원성산구,가음동


In [6]:
# 읍면동과 도로명 주소를 결합하여 새로운 주소 생성
edited_df = pd.DataFrame()
edited_df['edited_address'] = divided_df['읍면동'] + ' ' + df['도로명']
edited_df.head()

,edited_address
0,가음동 가양로52번길
1,가음동 가음로92번길
2,가음동 가음정로23번길
3,가음동 가음정로11번길
4,가음동 가양로34번길


In [7]:
result_df = pd.DataFrame(columns=['Address', 'Latitude', 'Longitude', 'Zip Code'])
result_df

,Address,Latitude,Longitude,Zip Code


In [ ]:
for idx, row in edited_df.iterrows():
    address = row['edited_address']
    if pd.isna(address):
        continue
    lat, lon = geocoding(address)
    print(f"Geocoded {address} to ({lat}, {lon})")
    if lat is None or lon is None:
        # Replace None with 0 so we still record the entry
        lat = 0 if lat is None else lat
        lon = 0 if lon is None else lon

    if 'convert_df' not in globals():
        convert_df = pd.DataFrame(columns=['address', 'lat', 'lon'])

    convert_df.loc[len(convert_df)] = [address, lat, lon]


Geocoded 가음동 가양로52번길 to (35.2056831, 128.7013065)
Geocoded 가음동 가음로92번길 to (35.2050142, 128.7007619)
Geocoded 가음동 가음정로23번길 to (None, None)
Geocoded 가음동 가음정로11번길 to (35.2110184, 128.6874138)
Geocoded 가음동 가양로34번길 to (35.2049314, 128.6994114)
Geocoded 가음동 가음로64번길 to (35.2065324, 128.701286)
Geocoded 가음동 가음로92번길 to (35.2050142, 128.7007619)
Geocoded 가음동 가양로 to (35.2027876, 128.6969623)
Geocoded 귀곡동 삼귀로 to (None, None)
Geocoded 귀산동 삼귀로466번길 to (None, None)
Geocoded 귀산동 귀산로60번길 to (None, None)
Geocoded 귀산동 삼귀로466번길 to (None, None)
Geocoded 귀산동 삼귀로466번길 to (None, None)
Geocoded 남양동 가음로57번길 to (None, None)
Geocoded 남양동 가음로101번길 to (None, None)
Geocoded 남양동 가음로57번길 to (None, None)
Geocoded 남양동 가음로57번길 to (None, None)
Geocoded 남양동 가음로57번길 to (None, None)
Geocoded 남양동 가음로101번길 to (None, None)
Geocoded 남양동 가음로101번길 to (None, None)
Geocoded 대방동 대암로101번길 to (None, None)
Geocoded 대방동 대암로115번길 to (None, None)
Geocoded 대방동 대암로105번길 to (35.2161744, 128.7083478)
Geocoded 대방동 대암로145번길 to (35.2140017, 128.7

## 2번째 작업 : Geocoder API 2.0 레퍼런스 기반 road_coordinate 확보

In [ ]:
#우선 결측치 및 이상치값을 포함한 데이터프레임을 csv로 임시 저장
convert_df.to_csv('II-1. Only_Geopy_result.csv', index=False, encoding='utf-8-sig')

In [ ]:
# convert_df에 nan값을 가진 행만 추출해서 df_nan에 저장
df_nan = convert_df[(convert_df['lat'] == 0) | (convert_df['lon'] == 0)]
df_nan.shape

In [ ]:
# df_nan에서 address컬럼에서 중복된 값 제거
df_drop_duplicated = df_nan.drop_duplicates(subset=['address'])
df_drop_duplicated.shape

In [ ]:
#df_drop_duplicated 대해 API를 이용해 재시도
import requests
apiurl = "https://api.vworld.kr/req/address?"
add_list, lat_list, lng_list = [], [], []
add_list, lat_list, lng_list

In [ ]:
for i in df_drop_duplicated.index:
    address = df_drop_duplicated['address'][i]
    print(address)
    # 주소에서 위도, 경도 좌표 검색
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "road",
        "key": "8B8F1EA3-89E3-3DF3-BF33-3318488BA5A4"
    }
    response = requests.get(apiurl, params=params)
    jsonData = None
    if response.status_code == 200:
        jsonData = response.json()
        if jsonData.get("response").get("status") == "OK":
            print(jsonData.get("response").get("result").get("point")) 
            lat = jsonData.get("response").get("result").get("point").get("y")
            lng = jsonData.get("response").get("result").get("point").get("x")
            add_list.append(address)
            lat_list.append(lat)
            lng_list.append(lng)
            print(f"Address: {address}, Latitude: {lat}, Longitude: {lng}")
        else:
            print(f"Error in response for address {address}: {jsonData.get('response').get('status')}")
    else:
        print(f"HTTP error {response.status_code} for address {address}")


In [ ]:
df_API = pd.DataFrame({'address': add_list, 'lat': lat_list, 'lon': lng_list})
df_API.shape


In [ ]:
df_API.to_csv('II-1. df_roadCoord_API.csv', index=False, encoding='utf-8-sig')

In [ ]:
    # coordinates_reverse = geocoding_reverse(lat, lon)
    # _, zip_code = coordinates_reverse if coordinates_reverse else (None, None)

    # result_df = pd.concat([result_df, pd.DataFrame({
    #     'Address': [address],
    #     'Latitude': [lat],
    #     'Longitude': [lon],
    #     'Zip Code': [zip_code]
    # })], ignore_index=True)
